## LLM Inference

Cells below show how to use the OllamaClient and the HFClient.

In [1]:
from inference import OllamaClient
client = OllamaClient(base_url="http://localhost:11434")
model="qwen2.5:0.5b-instruct"
client.download_model(model=model)
print(model)
client.warmup_model(model=model)

Model qwen2.5:0.5b-instruct downloaded successfully.
qwen2.5:0.5b-instruct
Model qwen2.5:0.5b-instruct warmed up successfully.


In [2]:
from finetune import PRewriteTrainer

trainer = PRewriteTrainer(
    model_id="gpt2",
    #model_id="Qwen/Qwen2.5-0.5B-Instruct", 
    dataset="openai/gsm8k:main",
    task="math",
    evaluator_model="qwen2.5:0.5b-instruct",
    ollama=True
)


Using Ollama evaluator
Model qwen2.5:0.5b-instruct warmed up successfully.
Loading model from local directory: models/gpt2


Map:   0%|          | 0/1319 [00:00<?, ? examples/s]

/home/raqkaaq/miniconda3/envs/cs401r/lib/python3.10/site-packages/trl/trainer/ppo_trainer.py:200: UserWarning: This trainer will soon be moved to trl.experimental and is a candidate for removal. If you rely on it and want it to remain, please share your comments here: https://github.com/huggingface/trl/issues/4223. Silence this warning by setting environment variable TRL_EXPERIMENTAL_SILENCE=1.
  warnings.warn(


AttributeError: 'AutoModelForCausalLMWithValueHead' object has no attribute 'base_model_prefix'

In [ ]:
trainer.train()

===training policy===


AttributeError: 'GPT2LMHeadModel' object has no attribute 'score'